In [1]:
# Import the QICK drivers and auxiliary libraries
from qick import *
# from qick_lib.qick import *
%pylab inline

# for now, all the tProc v2 classes need to be individually imported (can't use qick.*)

# the main program class
from qick.asm_v2 import AveragerProgramV2
# for defining sweeps
from qick.asm_v2 import QickSweep1D

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [40]:
from qick import *
from qick.pyro import make_proxy

soc, soccfg = make_proxy(ns_host="192.168.1.163", ns_port=8000, proxy_name="rfsoc")
print(soccfg)

Pyro.NameServer PYRO:Pyro.NameServer@0.0.0.0:8000
rfsoc PYRO:obj_fe9a5259092f4c8e895200c608c62172@192.168.1.163:36167
QICK running on ZCU216, software version 0.2.289

Firmware configuration (built Sat Sep 28 22:15:40 2024):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	16 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.838 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 0 is 0_228, on JHC1
	1:	axis_signal_gen_v6 - envelope memory 16384 samples (1.709 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 1 is 1_228, on JHC2
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (3.419 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 2 is 2_228, on JHC1
	3:	axis_signal_gen_v6 - envelope memory 16384 samples (1.709 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 3 i

In [55]:
###########
# Program
###########

class ContinuousPulse(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        res_ch = cfg['res_ch']
        
        self.declare_gen(ch=gen_ch, nqz=2)
        self.declare_gen(ch=res_ch, nqz=2)
        # self.declare_readout(ch=ro_ch, length=cfg['ro_len'], freq=cfg['freq'], gen_ch=gen_ch)
        # self.declare_gen(ch=gen_ch, nqz=2, mixer_freq=cfg['mixer_freq'])
        # tproc configured
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])
        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch)
        ramp_len = 0.2
        self.add_gauss(ch=gen_ch, name="ramp", sigma=ramp_len/10, length=ramp_len, even_length=True)
        
        self.add_pulse(ch=gen_ch, name="myconst", ro_ch=ro_ch, 
                       style="const", 
                       length=1, 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=cfg['gain'],
                      )
        
        self.add_pulse(ch=res_ch, name="myconst1", ro_ch=ro_ch, 
                       style="const", 
                       length=1, 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=cfg['gain'],
                      )
        
    def _body(self, cfg):
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'])
        self.pulse(ch = cfg['res_ch'],name="myconst1", t=2.5+0.02)
        self.trigger(pins=[1], t=0)
        self.pulse(ch=cfg['gen_ch'], name="myconst", t=0.02 + 0.5)
        
exp_config = {
    'gen_ch': 2,
    'res_ch': 3,
    'ro_ch': 0,
    'freq': 5000,
    'trig_time': 0,
    'ro_len': 1.9,
    'gain': 1.0, # 0.1,
    'mixer_freq': 4500,
     }

###################
# Run the Program
###################

# for i in range(100):
prog =ContinuousPulse(soccfg, reps=1, final_delay=0.5, cfg=exp_config)
iq_list = prog.acquire_decimated(soc, soft_avgs=1)

  0%|          | 0/1 [00:00<?, ?it/s]

# Continuous test

In [24]:
###########
# Program
###########

class ContinuousPulse(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=2)
        # self.declare_readout(ch=ro_ch, length=cfg['ro_len'], freq=cfg['freq'], gen_ch=gen_ch)
        # self.declare_gen(ch=gen_ch, nqz=2, mixer_freq=cfg['mixer_freq'])
        # tproc configured
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])
        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch)
                               
        ramp_len = 0.2
        self.add_gauss(ch=gen_ch, name="ramp", sigma=ramp_len/10, length=ramp_len, even_length=True)
        
        mode = ["oneshot", "periodic"][cfg["out_en"]]

        self.add_pulse(ch=gen_ch, name="myconst", ro_ch=ro_ch, 
                       style="const", 
                       length=1.0, 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=cfg['gain'],
                       mode=mode, stdysel="last"
                      )
    
    def _body(self, cfg):
        # self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'])
        self.trigger(ros=[cfg['ro_ch']], pins=[1], t=cfg['trig_time'], width= 0.5)
        self.pulse(ch=cfg['gen_ch'], name="myconst", t=0)
        
exp_config = {
    'gen_ch': 0,
    'ro_ch': 0,
    'freq': 4000,
    'trig_time': 0,
    'ro_len': 1.9,
    'gain': 0.1, # 0.1,
    'mixer_freq': 4500,
     }

###################
# Run the Program
###################

# prog =ContinuousPulse(soccfg, reps=1, final_delay=0.5, cfg=exp_config)
# iq_list = prog.acquire_decimated(soc, soft_avgs=1000)

def manualTune(out_en=1, **kwargs):
    exp_config.update(kwargs)
    config = {**exp_config, "out_en": out_en}
    prog =ContinuousPulse(soccfg, reps=1, final_delay=0.5, cfg=config)
    # iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=True, debug=False)
    iq_list = prog.acquire_decimated(soc, soft_avgs=1)
    if config["out_en"] == 0:
        print(dict(config), sort_dicts=False)


if __name__ == "__main__":
    manualTune(out_en=1, skewPhase=90, IQScale=1)

  0%|          | 0/1 [00:00<?, ?it/s]